<a href="https://colab.research.google.com/github/sbht04/mini-project/blob/main/Hello_Agent_CSV_FAQ_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain-experimental langchain-openai pandas requests==2.32.4
!pip install gdown

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5

In [4]:
import pandas as pd
import os
import sys
import gdown
from getpass import getpass
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# ==========================================
#  PART 1: AUTOMATIC FILE DOWNLOADER
# ==========================================

files_to_download = {
    "saas_docs.csv":         "https://drive.google.com/file/d/1RElOhN7bYsDAJUNQhYyqM7IzX-Xo6myq/view?usp=sharing",
    "credit_card_terms.csv": "https://drive.google.com/file/d/1_giivc_B0urOKpct0XY2yVZuxW3Eenuf/view?usp=sharing",
    "hospital_policy.csv":   "https://drive.google.com/file/d/1pL7OnDhnmz9pteIpBJ12gu2_ixrc2hPm/view?usp=sharing",
    "ecommerce_faqs.csv":    "https://drive.google.com/file/d/1O4fTjsLFbz55oOiwJUwLwZryO5OSSF6p/view?usp=sharing"
}

print("--- Downloading Files from Google Drive ---")
for filename, url in files_to_download.items():
    if not os.path.exists(filename):
        gdown.download(url, filename, quiet=False, fuzzy=True)
        print(f"Downloaded: {filename}")
    else:
        print(f"Skipped: {filename} (Already exists)")
print("--- Download Complete ---\n")


# ==========================================
#  PART 2: AI AGENT SETUP (MULTI-FILE)
# ==========================================

# 1. SETUP: Get API Key Securely
print("ENTER YOUR OPENAI API KEY BELOW:")
api_key = getpass()

# 2. LOAD ALL CSVs INTO A LIST
dataframes = [] # We will store all the loaded tables here
loaded_names = []

try:
    for filename in files_to_download.keys():
        df = pd.read_csv(filename)
        dataframes.append(df)
        loaded_names.append(filename)
        print(f"SUCCESS: Loaded '{filename}' ({len(df)} rows)")

except Exception as e:
    print(f"\nERROR loading files: {e}")
    sys.exit()

# 3. DEFINE THE RULES
system_prompt = """
You are a smart data assistant capable of reading multiple CSV files.
- You have access to 4 different datasets: SaaS Docs, Credit Card Terms, Hospital Policy, and Ecommerce FAQs.
- When asked a question, determine which DataFrame is most relevant.
- Do NOT answer from general knowledge.
- Answer in plain English.
"""

try:
    # ---------------------------------------------------------
    # TODO 1: Initialize the LLM
    # FIXED: Passed the api_key variable to ChatOpenAI
    # ---------------------------------------------------------
    llm = ChatOpenAI(
       model="gpt-4o-mini",
       temperature=0.0,
       api_key=api_key
    )


    # ---------------------------------------------------------
    # TODO 2: Create the Pandas Agent
    # ---------------------------------------------------------
    agent = create_pandas_dataframe_agent(
        llm,
        dataframes,
        verbose=True,
        agent_type="openai-functions",
        allow_dangerous_code=True
    )

    print("\nAI Agent is ready! You can ask questions across ALL files.")
    print("Example: 'What is the visiting hour in the hospital?' or 'What is the API limit?'")

except Exception as e:
    print(f"Error initializing agent: {e}")
    sys.exit()

--- Downloading Files from Google Drive ---
Skipped: saas_docs.csv (Already exists)
Skipped: credit_card_terms.csv (Already exists)
Skipped: hospital_policy.csv (Already exists)
Skipped: ecommerce_faqs.csv (Already exists)
--- Download Complete ---

ENTER YOUR OPENAI API KEY BELOW:
··········
SUCCESS: Loaded 'saas_docs.csv' (15 rows)
SUCCESS: Loaded 'credit_card_terms.csv' (15 rows)
SUCCESS: Loaded 'hospital_policy.csv' (15 rows)
SUCCESS: Loaded 'ecommerce_faqs.csv' (15 rows)

AI Agent is ready! You can ask questions across ALL files.
Example: 'What is the visiting hour in the hospital?' or 'What is the API limit?'


In [5]:
# ==========================================
#  PART 3: CHAT LOOP
# ==========================================
print("\nType 'exit' or 'quit' to stop conversation.\n")

while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "q"]:
        print("Goodbye!")
        break

    if not user_input:
        continue

    final_query = system_prompt + "\n\nQuestion: " + user_input
    print("AI is thinking...")

    try:
        # ---------------------------------------------------------
        # TODO 4: Invoke the Agent
        # Hint: Use agent.invoke() and pass the final_query
        # The result will be a dictionary, access ['output']
        # ---------------------------------------------------------

       result = agent.invoke({"input": final_query})
       response = result["output"]

        ## response = "..." # <--- REPLACE THIS WITH YOUR CODE

       print(f"AI: {response}\n" + "-"*30)
    except Exception as e:
        print(f"An error occurred: {e}")


Type 'exit' or 'quit' to stop conversation.

You: What is the visiting hour in the hospital?
AI is thinking...


> Entering new AgentExecutor chain...
An error occurred: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-abcde*****************************ef12. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
You: What is the visiting hour in the hospital?
AI is thinking...


> Entering new AgentExecutor chain...
An error occurred: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-abcde*****************************ef12. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}
You: exit
Goodbye!
